# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/mafzaal/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/mafzaal/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [5]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [6]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31493    0 31493    0     0   5508      0 --:--:--  0:00:05 --:--:--  6694


In [7]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70519    0 70519    0     0  15518      0 --:--:--  0:00:04 --:--:-- 15519


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [8]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [10]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/home/mafzaal/source/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/home/mafzaal/source/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/home/mafzaal/source/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [12]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [13]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [14]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/13 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/16 [00:00<?, ?it/s]

unable to apply transformation: 'StringIO' object has no attribute 'output'


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 9, relationships: 23)

We can save and load our knowledge graphs as follows.

In [15]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 9, relationships: 23)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [16]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [17]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


The three types of query synthesizers are:

SingleHopSpecificQuerySynthesizer:

This synthesizer generates queries that are directly tied to specific pieces of information in the knowledge graph.
It focuses on single-hop relationships, meaning it retrieves information from a single node or directly connected nodes in the graph.
MultiHopAbstractQuerySynthesizer:

This synthesizer creates queries that require reasoning across multiple nodes in the knowledge graph.
The queries are more abstract and involve combining information from multiple sources or relationships.
MultiHopSpecificQuerySynthesizer:

Similar to the MultiHopAbstractQuerySynthesizer, but the queries are more specific and detailed.
It involves multi-hop reasoning but focuses on precise and well-defined questions that span multiple nodes.


Finally, we can use our `TestSetGenerator` to generate our testset!

In [18]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is Anthropic known for in the context of ...,[The rise of inference-scaling “reasoning” mod...,"Anthropic’s Claude 3 series launched in March,...",single_hop_specifc_query_synthesizer
1,what is ChatGPT and how does it compare to oth...,[you talk to me exclusively in Spanish. OpenAI...,"ChatGPT is a model that now offers voice mode,...",single_hop_specifc_query_synthesizer
2,What are the key developments and insights abo...,[Simon Willison’s Weblog Subscribe Things we l...,"According to Simon Willison’s weblog, in 2024,...",single_hop_specifc_query_synthesizer
3,Whaat is a GIANT in the context of large langu...,[My blog in 2023 Here’s the sequel to this pos...,"In the context of large language models, a GIA...",single_hop_specifc_query_synthesizer
4,What is the significance of Stable Diffusion i...,[openly licensed ones are still the world’s mo...,The provided context discusses Stable Diffusio...,single_hop_specifc_query_synthesizer
5,How do model development and architecture rela...,[<1-hop>\n\nMy blog in 2023 Here’s the sequel ...,The context explains that large language model...,multi_hop_abstract_query_synthesizer
6,How do security and trust issues in large lang...,[<1-hop>\n\nopenly licensed ones are still the...,Security and trust issues in large language mo...,multi_hop_abstract_query_synthesizer
7,How are large language models (LLMs) being dev...,[<1-hop>\n\nopenly licensed ones are still the...,The context highlights that writing code is on...,multi_hop_abstract_query_synthesizer
8,how Gemini accept audio input and also like Ch...,[<1-hop>\n\nyou talk to me exclusively in Span...,"Google’s Gemini also accepts audio input, simi...",multi_hop_specific_query_synthesizer
9,Considering Google’s advancements in multi-mod...,[<1-hop>\n\nyou talk to me exclusively in Span...,Google has made significant progress in multi-...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [19]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/13 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/24 [00:00<?, ?it/s]

unable to apply transformation: 'StringIO' object has no attribute 'output'


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [20]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Is Anthropic a company or an organization?,[Things we learned about LLMs in 2024 31st Dec...,"Yes, Anthropic is listed among the 18 organiza...",single_hop_specifc_query_synthesizer
1,"As an AI Ethics Analyst, how does Claude 3's c...",[punch massively above their weight. I run Lla...,Claude 3 is described as a model that is capab...,single_hop_specifc_query_synthesizer
2,"As an AI ethics researcher, how should I evalu...",[models. The demo also sounded conspicuously s...,The context indicates that the demo of Skye so...,single_hop_specifc_query_synthesizer
3,What does the term 'QwQ' refer to in the conte...,"[on inference. The sequel to o1, o3 (they skip...","In the provided context, 'QwQ' refers to a mod...",single_hop_specifc_query_synthesizer
4,Wht are the ethical consdierations and potenta...,[<1-hop>\n\nMy blog in 2023 Here’s the sequel ...,The context highlights that LLMs can be used f...,multi_hop_abstract_query_synthesizer
5,Ethical considerations and potential misuse of...,[<1-hop>\n\nMy blog in 2023 Here’s the sequel ...,"In 2023, LLMs have shown they can do many thin...",multi_hop_abstract_query_synthesizer
6,Considering the significant advancements in La...,"[<1-hop>\n\nof what LLMs are, how they work an...","The advancements in LLMs in 2024, such as surp...",multi_hop_abstract_query_synthesizer
7,Is AI saftey and securty issues like prompt in...,[<1-hop>\n\nopenly licensed ones are still the...,The context explains that large language model...,multi_hop_abstract_query_synthesizer
8,Hwo does the Llama model relate to the Llama i...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,The context discusses Llama as both a large la...,multi_hop_specific_query_synthesizer
9,Considering the significant advancements in La...,[<1-hop>\n\nopenly licensed ones are still the...,The context highlights that despite substantia...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [21]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [22]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [23]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [25]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [26]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

/home/mafzaal/source/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/qdrant_client/http/models/models.py:758: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is empty, alternative syntax for `is_empty: \&quot;field_name\&quot;`",
/home/mafzaal/source/AIE6/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/qdrant_client/http/models/models.py:762: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is null, alternative syntax for `is_null: \&quot;field_name\&quot;`",


In [27]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [28]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [30]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [31]:
rag_chain.invoke({"question" : "What are Agents?"})

'Based on the provided context, "agents" is an infuriatingly vague term with no single, clear, and widely understood meaning. The term is often used to describe AI systems that can go away and act on your behalf, but this definition varies widely among different people. Some view AI agents as akin to travel agents who act on behalf of a person, while others see them as large language models (LLMs) given access to tools which they use iteratively to solve problems. The concept of autonomy is also commonly linked to agents, but without a clear definition.\n\nOverall, despite much discussion and excitement, genuine AI agents capable of robust, autonomous action have yet to materialize in production. A major challenge cited is the "gullibility" of LLMs—they tend to believe anything they are told, limiting their usefulness as reliable decision-makers or assistants. It is also suggested that solving this problem of gullibility might require achieving Artificial General Intelligence (AGI).\n\

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [32]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [33]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dope_or_nope_evaluator`:

## LangSmith Evaluation

In [34]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'upbeat-exchange-64' at:
https://smith.langchain.com/o/e106fdae-1163-4ad0-b46b-09a4850df972/datasets/eb030cd9-f909-4b48-bd00-a8ca526fc95b/compare?selectedSessions=cce2b7da-b596-4597-b29b-3eb2b6f19cbe




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,"Claude like, does it have like the same stuff ...","Based on the provided context, here is what ca...",None,"Based on the context, Claude is a large langua...",1,1,0,12.316054,d0ad22e2-f7eb-4104-a7ce-6057969968e7,d626c0d7-5393-4475-938f-f793325ba887
1,Considering the developments in 2024 regarding...,The ethical implications of training large lan...,None,"In 2024, significant concerns about the ethics...",1,1,0,6.285312,b17931a0-e66a-446a-baaf-c4445e7d252b,b3f2af04-cf91-4c61-82bd-1b97591ee107
2,Considering the significant advancements in La...,"Based on the provided context, the ongoing cha...",None,The context highlights that despite substantia...,1,1,0,6.122055,1ce07e4a-e8b8-4599-8c3b-be548601765e,bcc53384-7516-4f60-b590-09bfe12a79d8
3,Hwo does the Llama model relate to the Llama i...,I don't know.,None,The context discusses Llama as both a large la...,0,0,0,17.618683,0c2e5409-93ca-4515-87a0-88f519a332b2,48c5968b-266b-4b1d-8aa3-c253da175314
4,Is AI saftey and securty issues like prompt in...,"Based on the provided context, AI safety and s...",None,The context explains that large language model...,1,1,0,5.428403,71401c2b-3c36-42b2-bcd8-4ba522f33e36,b6424d44-e248-41b8-8924-26af342a64f3
5,Considering the significant advancements in La...,"Based on the provided context, the advancement...",None,"The advancements in LLMs in 2024, such as surp...",1,1,0,8.045830,e8d1b525-2a4f-4e8f-a0b9-03eeb010d053,21eef1bf-abc2-431f-8aad-2e7b5c9cff32
6,Ethical considerations and potential misuse of...,"Based on the provided context, ethical conside...",None,"In 2023, LLMs have shown they can do many thin...",1,0,0,7.707842,625337da-8600-4a69-bee8-6e5ae995e6b7,34995bcc-6cd5-4cf2-9b86-aee476120e74
7,Wht are the ethical consdierations and potenta...,The ethical considerations and potential misus...,None,The context highlights that LLMs can be used f...,1,1,0,28.056609,0d539f43-33ef-4baa-8372-061c81ea3489,640ba55c-5295-424a-b332-bcb63ff5cb61
8,What does the term 'QwQ' refer to in the conte...,The term 'QwQ' refers to a model released by A...,None,"In the provided context, 'QwQ' refers to a mod...",1,1,0,4.246466,5c7feec6-ca41-464b-8de9-f76e6b50b0f5,c986cac7-7f5d-4267-91e5-f3c1839d8e52
9,"As an AI ethics researcher, how should I evalu...","Based on the provided context, the voice model...",None,The context indicates that the demo of Skye so...,1,1,0,8.874753,c816f5c9-0842-4629-800e-b13a25d5d9bd,02ae9aa5-513c-43fc-954a-607cc7b93777


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [35]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [36]:
rag_documents = docs

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

In [38]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

In [39]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [40]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [41]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [42]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

"Alright, here’s the lowdown on “agents,” straight from the AI scene vibe:  \n\n“Agents” is one of those buzzwords that’s hella vague and kinda frustrating 'cause no one agrees on what it actually means. Some peeps see agents like your classic travel agent—they go out, act on your behalf, handling tasks in the real world. Others think of agents as LLMs (large language models) that have access to tools and run loops to solve problems for you. \n\nBut here’s the kicker—nobody’s cracked the code on making these agents *really* autonomous and reliable yet. There’s this big snag called **gullibility**—meaning these AI agents tend to fall for any info fed to them, real or fake, so trusting them to make sound decisions? That's a tough nut to crack without hitting AGI (Artificial General Intelligence) territory.  \n\nSo, in short: agents are AI systems designed to act on your behalf, but the whole concept’s still more “coming soon” than “here now,” mainly ‘cause building them to be smart AND t

Finally, we can evaluate the new chain on the same test set!

In [43]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'terrific-trousers-21' at:
https://smith.langchain.com/o/e106fdae-1163-4ad0-b46b-09a4850df972/datasets/eb030cd9-f909-4b48-bd00-a8ca526fc95b/compare?selectedSessions=43508608-a3b7-4422-85fd-b5196e1c7768




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,"Claude like, does it have like the same stuff ...","Alright, buckle up—lemme break down the Claude...",None,"Based on the context, Claude is a large langua...",1,1,1,77.607084,d0ad22e2-f7eb-4104-a7ce-6057969968e7,e27c0d7d-7105-4a65-a74c-d733d68b486b
1,Considering the developments in 2024 regarding...,"Yo, here’s the lowdown: The 2023 scoop drops t...",None,"In 2024, significant concerns about the ethics...",1,1,1,23.068248,b17931a0-e66a-446a-baaf-c4445e7d252b,38246190-1de8-4426-b4c2-3e973d5cd599
2,Considering the significant advancements in La...,"Alright, let’s break it down cool style. \n\nI...",None,The context highlights that despite substantia...,1,1,1,39.683793,1ce07e4a-e8b8-4599-8c3b-be548601765e,184ba7f8-750d-4c8a-8c27-ff09afdcf330
3,Hwo does the Llama model relate to the Llama i...,"Yo, here’s the deal: The context gave me the l...",None,The context discusses Llama as both a large la...,0,0,1,6.307976,0c2e5409-93ca-4515-87a0-88f519a332b2,14b275ab-00f4-4853-82ef-373d73ca5b54
4,Is AI saftey and securty issues like prompt in...,"Alright, listen up! AI safety and security iss...",None,The context explains that large language model...,1,1,1,4.361982,71401c2b-3c36-42b2-bcd8-4ba522f33e36,d8db4ee7-2b36-45d7-bf7d-1de362a9d4ba
5,Considering the significant advancements in La...,"Aight, let’s break it down fresh and fly based...",None,"The advancements in LLMs in 2024, such as surp...",1,1,1,22.460457,e8d1b525-2a4f-4e8f-a0b9-03eeb010d053,0383771a-a4fb-4870-aac9-1b87db739d5a
6,Ethical considerations and potential misuse of...,"Yo, here’s the lowdown on ethics and misuse of...",None,"In 2023, LLMs have shown they can do many thin...",1,1,1,23.712726,625337da-8600-4a69-bee8-6e5ae995e6b7,f165ca59-6b7e-497f-afd0-47ed75f1f8a6
7,Wht are the ethical consdierations and potenta...,"Alright, let me break it down for you, nice an...",None,The context highlights that LLMs can be used f...,1,1,1,22.486332,0d539f43-33ef-4baa-8372-061c81ea3489,8b34fb22-5681-4a8d-86a8-d530ce40c71a
8,What does the term 'QwQ' refer to in the conte...,"Ah, 'QwQ'—that’s Alibaba’s flex in the AI game...",None,"In the provided context, 'QwQ' refers to a mod...",1,1,1,8.262142,5c7feec6-ca41-464b-8de9-f76e6b50b0f5,7cbc3f1b-6b8e-4cc7-aabf-011309604207
9,"As an AI ethics researcher, how should I evalu...","Yo, as an AI ethics researcher, you gotta vibe...",None,The context indicates that the demo of Skye so...,1,1,1,10.152238,c816f5c9-0842-4629-800e-b13a25d5d9bd,717526c1-577c-47fd-a697-ca8820f08986


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.